In [1]:
import os
import pyspark
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType
from pyspark.sql.functions import hour, when, col, date_format, to_timestamp, regexp_replace
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import numpy as np
from sklearn import decomposition
from sklearn.linear_model import LinearRegression

In [2]:
IID = '3.b.2'
SeriesCode = 'SH_STA_AIRP'

In [3]:
def to_explode(df, by):

    # Filter dtypes and split into column names and type description
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))
    # Spark SQL supports only homogeneous columns
    assert len(set(dtypes)) == 1, "All columns have to be of the same type"

    # Create and explode an array of (column_name, column_value) structs
    kvs = F.explode(F.array([
      F.struct(F.lit(c).alias("Year"), col(c).cast("double").alias("Value")) for c in cols
    ])).alias("kvs")

    return df.select(by + [kvs]).select(by + ["kvs.Year", "kvs.Value"])

In [4]:
spark = SparkSession.builder.master("local[2]").appName("project").config("spark.driver.memory", "9g").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [5]:
##### Read WDI data
df_WDI = sqlContext.read.option("header",True).csv("WDIData.csv").drop("_c65")
df_WDI = df_WDI.withColumn("Indicator Code Merge", regexp_replace(col("Indicator Code"), "\\.", ""))

##### Read SDG Goal3 data
df_SDG = sqlContext.read.option("header",True).csv("Goal3.csv").drop("_c51")

In [6]:
##### Preprocesse WDI data
if not os.path.exists('WDI_statistics.csv'):
    df_WDI_stats = to_explode(df_WDI, ["Country Name","Country Code", "Indicator Name", "Indicator Code"])
    df_WDI_stats = df_WDI_stats.na.drop(subset=["Value"]) \
                                .groupBy("Indicator Name", "Indicator Code") \
                                .count() \
                                .sort(col("count").desc())

    df_WDI_stats.repartition(1).write.option("header", "true").csv('WDI_statistics.csv')
else:
    ##### Read WDI saved statistics data
    df_WDI_stats = sqlContext.read.option("header",True).csv("WDI_statistics.csv")

df_WDI_stats.show(5)

+--------------------+-----------------+-----+
|      Indicator Name|   Indicator Code|count|
+--------------------+-----------------+-----+
|   Population, total|      SP.POP.TOTL|15671|
|Rural population ...|   SP.RUR.TOTL.ZS|15652|
|Urban population ...|SP.URB.TOTL.IN.ZS|15652|
|    Rural population|      SP.RUR.TOTL|15619|
|    Urban population|      SP.URB.TOTL|15619|
+--------------------+-----------------+-----+
only showing top 5 rows



In [7]:
##### Find countries which exists in both dataframe
countriesSDG = df_SDG.select("GeoAreaName").distinct().sort("GeoAreaName")
countriesWDI = df_WDI.select("Country Name").distinct().sort("Country Name")
countries = countriesSDG.intersect(countriesWDI).sort("GeoAreaName").collect()
countries = [v["GeoAreaName"] for v in countries]

In [ ]:
##### Get list of WDIs
indicators = df_WDI.groupBy("Indicator Name", "Indicator Code").count()

##### Select specific SDGI
df_SDG_filter = df_SDG.where(col("SeriesCode")==SeriesCode) \
                        .where(col("GeoAreaName").isin(countries)) \
                        .where(col("Value")!="NaN") \
                        .select(["Goal","Target","Indicator","SeriesCode","SeriesDescription","GeoAreaCode","GeoAreaName","TimePeriod","Value","Age","Sex"])

##### Additional filters if required
# df_SDG_filter = df_SDG_filter.where((col("Age")=="BOTHSEX"))  
# df_SDG_filter = df_SDG_filter.where((col("Sex")=="BOTHSEX"))  

print('\n SDG Indicator data:')
df_SDG_filter.show(5)
print("Check SeriesCode:", df_SDG_filter.select("Sex").distinct().collect())
print("Check SeriesCode:", df_SDG_filter.select("Age").distinct().collect())

##### Focus only on WDI indicators with sufficient data
df_WDI_stats_filter = df_WDI_stats.withColumn("count", col("count").cast("int")).where(col("count")>2000)
WDIs = df_WDI_stats_filter.select("Indicator Name").collect()
WDIs = [v["Indicator Name"] for v in WDIs]
print('\n Number of WDI data:')
print(len(WDIs))

##### Join two dataframes
years = df_SDG_filter.select("TimePeriod").distinct().sort("TimePeriod").collect()
years = [v["TimePeriod"] for v in years]

df_WDI_filter = df_WDI.select(["Country Name","Indicator Name"]+years)
df_WDI_filter = to_explode(df_WDI_filter, ["Country Name","Indicator Name"])

df_join = df_WDI_filter.withColumnRenamed("Value", "WDI Value") \
                        .join(df_SDG_filter, \
                            (col("Year")==col("TimePeriod")) & (col("Country Name")==col("GeoAreaName")),"inner") \
                        .withColumnRenamed("Value", "SDG Value") \
                        .where(col("WDI Value").isNotNull()) \
                        .where(col("SDG Value").isNotNull()) \
                        .withColumn("SDG Value", col("SDG Value").cast("double")) \
                        .withColumn("WDI Value", col("WDI Value").cast("double")) \
                        .sort("Country Name", "Indicator Name", "Year") \
                        .select("Country Name", "Indicator Name", "Year", "WDI Value", "SDG Value")
print('\n Combined SDG and WDI data:')
df_join.show(5, truncate=False)


 SDG Indicator data:
+----+------+---------+-----------+--------------------+-----------+-----------+----------+-----+----+----+
|Goal|Target|Indicator| SeriesCode|   SeriesDescription|GeoAreaCode|GeoAreaName|TimePeriod|Value| Age| Sex|
+----+------+---------+-----------+--------------------+-----------+-----------+----------+-----+----+----+
|   3|   3.9|    3.9.1|SH_STA_AIRP|Crude death rate ...|          1|      World|      2016|   95|null|null|
|   3|   3.9|    3.9.1|SH_STA_AIRP|Crude death rate ...|          4|Afghanistan|      2016|   95|null|null|
|   3|   3.9|    3.9.1|SH_STA_AIRP|Crude death rate ...|          8|    Albania|      2016|  105|null|null|
|   3|   3.9|    3.9.1|SH_STA_AIRP|Crude death rate ...|         12|    Algeria|      2016|   40|null|null|
|   3|   3.9|    3.9.1|SH_STA_AIRP|Crude death rate ...|         24|     Angola|      2016|   67|null|null|
+----+------+---------+-----------+--------------------+-----------+-----------+----------+-----+----+----+
only s

In [ ]:
##### Calculate correlations
print('\n Calculating correlation:')
with open('correlation_{}.csv'.format(SeriesCode), 'w') as f:
    f.write("%s,%s,%s,%s\n"%('WDI','DataCount','Value','abs'))

    for i, indicator in enumerate(WDIs):
        print(i, indicator)

        temp = df_join.where(col("Indicator Name")==indicator) 

        count = temp.count()
        if count == 0:
            print("No WDI data")
            corr = 0
        else:
            corr = temp.stat.corr("SDG Value", "WDI Value")
            print("Data count:{}, Corr:{}".format(count, corr))
        
        f.write('"%s",%s,%s,%s\n'%(indicator,count,corr,abs(corr)))


 Calculating correlation:
0 Population, total
Data count:160, Corr:0.11289945738209896
1 Rural population (% of total population)
Data count:160, Corr:0.5029078645688483
2 Urban population (% of total population)
Data count:160, Corr:-0.5029078645688484
3 Rural population
Data count:160, Corr:0.1342041086797362
4 Urban population
Data count:160, Corr:0.09363557523170388
5 Population growth (annual %)
Data count:160, Corr:-0.03933160461125266
6 Urban population growth (annual %)
Data count:160, Corr:0.127938059551726
7 Land area (sq. km)
Data count:159, Corr:0.06115840501326574
8 Surface area (sq. km)
Data count:159, Corr:0.06074336158817552
9 Population density (people per sq. km of land area)
Data count:161, Corr:-0.11002902125621584
10 Rural population growth (annual %)
Data count:158, Corr:0.13716275270383707
11 Birth rate, crude (per 1,000 people)
Data count:161, Corr:0.3947212957900788
12 Death rate, crude (per 1,000 people)
Data count:161, Corr:0.5624539721050158
13 Population a

Data count:159, Corr:0.1894620333435029
93 CO2 emissions from liquid fuel consumption (kt)
Data count:159, Corr:0.0609370938253692
94 CO2 emissions (kt)
Data count:159, Corr:0.08903766469768747
95 CO2 emissions (metric tons per capita)
Data count:159, Corr:-0.41779886817752243
96 Arable land (% of land area)
Data count:159, Corr:0.26651935887562006
97 Primary education, duration (years)
Data count:161, Corr:-0.2225993359800757
98 Food production index (2004-2006 = 100)
Data count:157, Corr:0.19174691056859633
99 Livestock production index (2004-2006 = 100)
Data count:157, Corr:-0.05749241988897961
100 Secondary education, duration (years)
Data count:161, Corr:0.25067788037059824
101 Crop production index (2004-2006 = 100)
Data count:157, Corr:0.2705767390757292
102 Merchandise exports by the reporting economy, residual (% of total merchandise exports)
Data count:161, Corr:-0.11992062775323445
103 Merchandise exports by the reporting economy (current US$)
Data count:161, Corr:0.04060238

Data count:159, Corr:-0.12850583108410235
176 Mortality rate, neonatal (per 1,000 live births)
Data count:161, Corr:0.5393345246367782
177 Adjusted savings: mineral depletion (% of GNI)
Data count:157, Corr:0.10866668879608231
178 Domestic credit to private sector by banks (% of GDP)
Data count:151, Corr:-0.41916825929857054
179 Monetary Sector credit to private sector (% GDP)
Data count:151, Corr:-0.41922595780288546
180 Foreign direct investment, net inflows (% of GDP)
Data count:156, Corr:-0.11778333104639321
181 Primary education, pupils
Data count:134, Corr:0.12721357258206925
182 Claims on central government, etc. (% GDP)
Data count:151, Corr:-0.04047139692907622
183 Number of neonatal deaths
Data count:161, Corr:0.13966504826704715
184 Adjusted savings: consumption of fixed capital (% of GNI)
Data count:157, Corr:-0.12248056336037218
185 Adjusted savings: carbon dioxide damage (% of GNI)
Data count:157, Corr:0.13800587129268602
186 GDP (current LCU)
Data count:155, Corr:0.021932

Data count:134, Corr:0.19035027192352408
266 School enrollment, secondary (% gross)
Data count:108, Corr:-0.5897953121748343
267 Net primary income (Net income from abroad) (current LCU)
Data count:154, Corr:-0.02653468849020543
268 Net primary income (Net income from abroad) (current US$)
Data count:154, Corr:-0.049245979863416486
269 Foreign direct investment, net outflows (% of GDP)
Data count:144, Corr:-0.12568892707425647
270 Grants, excluding technical cooperation (BoP, current US$)
Data count:115, Corr:0.01740319195248227
271 Immunization, measles (% of children ages 12-23 months)
Data count:161, Corr:-0.5313172691588243
272 Ores and metals exports (% of merchandise exports)
Data count:135, Corr:0.06257067115165632
273 Industry (including construction), value added (constant 2010 US$)
Data count:148, Corr:0.06273212943673914
274 Personal remittances, received (current US$)
Data count:152, Corr:0.09256358094774579
275 Arms imports (SIPRI trend indicator values)
Data count:107, Co

No WDI data
355 Exports of goods and services (annual % growth)
Data count:136, Corr:0.10619915965661168
356 Final consumption expenditure (constant 2010 US$)
Data count:131, Corr:0.040456873427845666
357 Services, value added (constant 2010 US$)
Data count:148, Corr:0.04235494469842025
358 Total reserves in months of imports
Data count:143, Corr:-0.046693623883708804
359 Secondary income, other sectors, payments (BoP, current US$)
Data count:148, Corr:0.020729506482355213
360 Personal remittances, received (% of GDP)
Data count:150, Corr:0.2584025005275721
361 Gross value added at basic prices (GVA) (constant 2010 US$)
Data count:139, Corr:0.03729540736906439
362 Households and NPISHs Final consumption expenditure per capita (constant 2010 US$)
Data count:130, Corr:-0.549809233817364
363 Households and NPISHs Final consumption expenditure (constant 2010 US$)
Data count:130, Corr:0.043804956669497734
364 Net official flows from UN agencies, UNDP (current US$)
Data count:114, Corr:0.086

Data count:142, Corr:-0.18487320548024963
442 Export value index (2000 = 100)
Data count:157, Corr:0.27024484633010665
443 Adjusted savings: gross savings (% of GNI)
Data count:137, Corr:-0.1549466076323643
444 Import value index (2000 = 100)
Data count:157, Corr:0.3885979460645246
445 Adjusted savings: net national savings (% of GNI)
Data count:137, Corr:-0.07522946910418221
446 Industry (including construction), value added (constant LCU)
Data count:150, Corr:0.018924589204880837
447 Gross savings (% of GDP)
Data count:138, Corr:-0.13653974319896126
448 GNI per capita growth (annual %)
Data count:132, Corr:0.03322614247669434
449 GNI growth (annual %)
Data count:132, Corr:0.007096755129319928
450 School enrollment, preprimary, female (% gross)
Data count:114, Corr:-0.3860093685439854
451 School enrollment, preprimary, male (% gross)
Data count:114, Corr:-0.37626749166668216
452 Repeaters, primary, total (% of total enrollment)
Data count:112, Corr:0.22314054146006673
453 GNI per capi

Data count:157, Corr:-0.2858548646067644
519 Employment in agriculture, male (% of male employment) (modeled ILO estimate)
Data count:157, Corr:0.5933934494561315
520 Net primary income (BoP, current US$)
Data count:149, Corr:-0.06971147288774951
521 Employment to population ratio, ages 15-24, male (%) (modeled ILO estimate)
Data count:157, Corr:-0.15553944939874462
522 Self-employed, female (% of female employment) (modeled ILO estimate)
Data count:157, Corr:0.5757487912265468
523 Self-employed, male (% of male employment) (modeled ILO estimate)
Data count:157, Corr:0.5944760124710204
524 Reserves and related items (BoP, current US$)
Data count:149, Corr:-0.15072737470603073
525 Net errors and omissions (BoP, current US$)
Data count:149, Corr:-0.12767570338473247
526 Net financial account (BoP, current US$)
Data count:149, Corr:-0.06601761636686838
527 Gross value added at basic prices (GVA) (constant LCU)
Data count:141, Corr:0.021139387816434665
528 Foreign direct investment, net (B

Data count:130, Corr:0.015144977003850302
605 GNI per capita (constant LCU)
Data count:130, Corr:-0.04003963089907951
606 Debt service on external debt, total (TDS, current US$)
Data count:102, Corr:-0.013235040822701225
607 PPG, official creditors (NFL, current US$)
Data count:102, Corr:-0.11622537065669757
608 Debt service on external debt, public and publicly guaranteed (PPG) (TDS, current US$)
Data count:102, Corr:-0.11489908171326117
609 Agriculture, forestry, and fishing, value added per worker (constant 2010 US$)
Data count:146, Corr:-0.17688964753522388
610 IBRD loans and IDA credits (DOD, current US$)
Data count:103, Corr:0.08471199504385185
611 Import volume index (2000 = 100)
Data count:157, Corr:0.2780267852853624
612 Net financial flows, multilateral (NFL, current US$)
Data count:102, Corr:-0.16072478307862476
613 Import unit value index (2000 = 100)
Data count:156, Corr:0.14539660067912766
614 Net barter terms of trade index (2000 = 100)
Data count:157, Corr:0.03929149956

Data count:102, Corr:-0.43327186798324435
689 Machinery and transport equipment (% of value added in manufacturing)
Data count:86, Corr:-0.09514792259783915
690 Social contributions (% of revenue)
Data count:116, Corr:-0.1319731503088974
691 Other taxes (% of revenue)
Data count:116, Corr:-0.21933328686028494
692 Tertiary education, academic staff (% female)
Data count:76, Corr:-0.02750784027106325
693 Taxes on income, profits and capital gains (% of revenue)
Data count:116, Corr:-0.23890549789300228
694 Taxes on international trade (% of revenue)
Data count:116, Corr:0.2940655906310205
695 ICT service exports (% of service exports, BoP)
Data count:141, Corr:0.22346246512967413
696 Government expenditure on education, total (% of GDP)
Data count:92, Corr:-0.4987370163820588
697 Taxes on goods and services (% of revenue)
Data count:115, Corr:0.21718503589055121
698 PPG, IDA (DOD, current US$)
Data count:80, Corr:0.23291627205433865
699 Newborns protected against tetanus (%)
Data count:9

Data count:122, Corr:0.10642396736026961
774 Prevalence of HIV, male (% ages 15-24)
Data count:105, Corr:-0.01955935249586498
775 Prevalence of HIV, female (% ages 15-24)
Data count:105, Corr:-0.02066719734367252
776 Women's share of population ages 15+ living with HIV (%)
Data count:105, Corr:0.3109615345818723
777 Share of tariff lines with international peaks, all products (%)
Data count:122, Corr:0.2150365139181479
778 Tariff rate, applied, weighted mean, all products (%)
Data count:122, Corr:0.22677919104388541
779 Tariff rate, most favored nation, weighted mean, all products (%)
Data count:122, Corr:0.24888778924779736
780 Tariff rate, applied, simple mean, all products (%)
Data count:122, Corr:0.24825856666262938
781 School enrollment, secondary (% net)
Data count:89, Corr:-0.45057288576324467
782 Incidence of HIV, ages 15-49 (per 1,000 uninfected population ages 15-49)


In [12]:
##### Filter similar indicators
indicator_codes = indicators.select("Indicator Code").collect()
indicator_codes = np.array([v["Indicator Code"] for v in indicator_codes])
indicator_codes_filter = []
for i in sorted(indicator_codes):
    if '.FE' in i:
        if '.FE.IN' in i:
            if i.replace('.FE.IN','.TO') in indicator_codes:
#                 print(i, 'REMOVED')
                continue
            elif i.replace('.FE.IN','') in indicator_codes:
#                 print(i, 'REMOVED')
                continue
        if i.replace('.FE','') in indicator_codes:
#             print(i, 'REMOVED')
            continue
    elif '.MA' in i:
        if '.MA.IN' in i:
            if i.replace('.MA.IN','.TO') in indicator_codes:
#                 print(i, 'REMOVED')
                continue
            elif i.replace('.MA.IN','') in indicator_codes:
#                 print(i, 'REMOVED')
                continue
        if i.replace('.MA','') in indicator_codes:
#             print(i, 'REMOVED')
            continue
    
#     print(i, 'SELECTED')
    indicator_codes_filter.append(str(i).replace(".",""))

In [13]:
indicators = indicators.withColumn("Indicator Code", regexp_replace(col("Indicator Code"), "\\.", ""))

if os.path.exists('ranks.csv'):
    f = open('ranks.csv', 'a')
else:
    f = open('ranks.csv', 'w')
    f.write("%s,%s,%s,%s,%s,%s\n"%('SDG Indicator','Rank','Correlation Rank','Relative Rank','Weight','WDI'))

##### Read WDI saved correlation data
df_WDI_corr = sqlContext.read.option("header",True).csv("correlation_{}.csv".format(SeriesCode))

##### Focus only on highly correlated WDI indicators
df_WDI_corr_filter = df_WDI_corr.join(indicators, col("Indicator Name") == col("WDI"), "left") \
                                .where(col("Indicator Code").isin(indicator_codes_filter)) \
                                .where(col("abs") < 0.98) \
                                .withColumn("abs", col("abs").cast("double")).sort(col("abs").desc())
print('\n Corrleation data:')
df_WDI_corr_filter.select(["WDI","abs","Indicator Code"]).show(5, truncate = False)

indicators_all = df_WDI_corr_filter.select(["Indicator Code","Indicator Name"]).limit(20).collect()
indicators_code = [v["Indicator Code"] for v in indicators_all]
indicators_names = [v["Indicator Name"] for v in indicators_all]

df_WDI_filter = df_WDI.select(["Country Name","Indicator Name","Indicator Code Merge"]+years) \
                        .filter(col("Indicator Code Merge").isin(indicators_code))
df_WDI_filter = to_explode(df_WDI_filter, ["Country Name","Indicator Name","Indicator Code Merge"])

##### Pivot table to have Indicators as features
new_df = df_WDI_filter.groupby("Country Name","Year").pivot("Indicator Code Merge").agg(F.max("Value").alias("Value"))
matrix = np.array(new_df.na.drop().select(indicators_code).collect())
print('\n Data size for PCA:')
print(matrix.shape)

##### PCA decomposition on selected WDIs
pca = decomposition.PCA(n_components=5, random_state=6242)
pca.fit(matrix)
PCAmatrix = pca.transform(matrix)

##### PCA transform on data before fitting regression model
new_df = new_df.join(df_SDG_filter.select(["TimePeriod","GeoAreaName","Value"]), (col("Year")==col("TimePeriod")) & (col("Country Name")==col("GeoAreaName")), "inner")
matrixF = np.array(new_df.na.drop().select(indicators_code + ['Value']).collect())
print('\n Data size for Regression model:')
print(matrixF.shape)
X = pca.transform(matrixF[:,:-1])
y = matrixF[:,-1].astype(float)

##### Fitting regression model
reg = LinearRegression()
reg.fit(X,y)

##### Convert weights in terms of WDI
print('\n Recommended WDIs')
weight = np.identity(20)
weight = reg.predict(pca.transform(weight))
correlation_rank = np.arange(20)
regression_rank = np.argsort(np.abs(weight))[::-1]
relative_rank = correlation_rank + regression_rank
for i,(r1,r2) in enumerate(zip(relative_rank,regression_rank)):
    f.write('%s,%s,%s,%s,%s,"%s"\n'%(SeriesCode,i,r2,r1,abs(weight[r2]),indicators_names[r2]))
    print("{:02d} - {:.6f} - {}".format(r1, abs(weight[r2]), indicators_names[r2]))
    
f.close()


 Corrleation data:
+------------------------------------------------------------+------------------+---------------+
|WDI                                                         |abs               |Indicator Code |
+------------------------------------------------------------+------------------+---------------+
|Literacy rate, adult total (% of people ages 15 and above)  |0.7713618316766852|SEADTLITRZS    |
|Literacy rate, adult female (% of females ages 15 and above)|0.7681221104308069|SEADTLITRFEZS  |
|Literacy rate, adult male (% of males ages 15 and above)    |0.7637294178737252|SEADTLITRMAZS  |
|Literacy rate, youth male (% of males ages 15-24)           |0.7355329012521   |SEADT1524LTMAZS|
|Literacy rate, youth total (% of people ages 15-24)         |0.7244084566833092|SEADT1524LTZS  |
+------------------------------------------------------------+------------------+---------------+
only showing top 5 rows


 Data size for PCA:
(69, 20)

 Data size for Regression model:
(30, 21)
